In [ ]:
import os
import base64
import json
import time

from tqdm import tqdm
from openai import AzureOpenAI, BadRequestError
from mimetypes import guess_type

In [ ]:
api_base = ' '
api_key= ' '
deployment_name = "gpt-4o"
api_version = "2024-05-01-preview" # this might change in the future

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=api_base,
)

In [ ]:
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"

In [ ]:
# RENAME FILES FOR THE DISEASES

folder_path = './facial_diseases'

for subfolder in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder)

    if os.path.isdir(subfolder_path):
        count = 1
        for filename in os.listdir(subfolder_path):
            if filename.lower().endswith('.jpg'):
                # Example: ac100_JPG.rf.eff5eeb00d8883343f97e028dace935f.jpg
                new_name = f'{subfolder}_{count}.jpg'
                count +=1

                old_path = os.path.join(subfolder_path, filename)
                new_path = os.path.join(subfolder_path, new_name)
                os.rename(old_path, new_path)



"\n# Specify the path of the folder you want to open\nfolder_path = './facial_diseases'\n\nfor subfolder in os.listdir(folder_path):\n    subfolder_path = os.path.join(folder_path, subfolder)\n    \n    if os.path.isdir(subfolder_path):\n        count = 1\n        for filename in os.listdir(subfolder_path):\n            if filename.lower().endswith('.jpg'):\n                # Example: ac100_JPG.rf.eff5eeb00d8883343f97e028dace935f.jpg\n                new_name = f'{subfolder}_{count}.jpg'\n                count +=1\n\n                old_path = os.path.join(subfolder_path, filename)\n                new_path = os.path.join(subfolder_path, new_name)\n                \n                # Rename the file\n                os.rename(old_path, new_path)\n"

In [ ]:
example_json = {
    'description': '',
    'q_a' : [
        {
            'question': '',
            'answer' : '',
        },
        {
            'question': '',
            'answer' : '',
        },
        {
            'question': '',
            'answer' : '',
        },
        {
            'question': '',
            'answer' : '',
        },
        {
            'question': '',
            'answer' : '',
        }
    ]
}

In [ ]:
def get_description(prompt, data_url):
    response = client.chat.completions.create(
        model=deployment_name,
        response_format={'type':'json_object'},
        messages=[
            { "role": "system", "content": "Provide valid JSON output. The data scheme should be like this: " +  json.dumps(example_json)},
            { "role": "user", "content": [
                {
                    "type": "text",
                    "text": prompt
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": data_url
                    }
                }
            ] }
        ],
        max_tokens=2000
    )

    msg = response.choices[0].message.content
    data = json.loads(msg)

    return data

In [ ]:
folder_path = './facial_diseases'

for subfolder in os.listdir(folder_path):
    subfolder_path = os.path.join(folder_path, subfolder)

    if os.path.isdir(subfolder_path):

        image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith('.jpg')]
        json_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith('.json')]

        for filename in tqdm(image_files, desc=f"{subfolder}", ncols=100):
            # since photos are .jpg I used :-4, if they are .jpeg use :-5
            if f'{filename[:-4]}.json' not in json_files:
                try:
                    img_path = os.path.join(subfolder_path, filename)
                    img_data = local_image_to_data_url(img_path)

                    prompt = (
                        f"Provide a valid JSON output. You know that the provided image contains {subfolder} disease. "
                        "Provide a detailed description of the image and create 5 questions with their answers about the description."
                    )

                    data = get_description(prompt=prompt, data_url=img_data)
                    json_path = os.path.join(subfolder_path, f'{filename[:-4]}.json')

                    with open(json_path, 'w') as f:
                        json.dump(data, f, indent=4)

                    time.sleep(60)

                # sometimes images are labeled as sexual content idk why
                except BadRequestError as e:
                    continue


rosacea: 100%|██████████████████████████████████████████████████| 100/100 [1:01:23<00:00, 36.84s/it]


In [ ]:
acne_data = {
    "description": "The image displays a close-up view of human skin affected by acne. It features an area of inflammation, redness, and the presence of a lesion that appears raised and swollen. Surrounding the affected area, the skin texture is highlighted with visible pores, indicating possible acne-prone skin. The central lesion suggests a pustule or nodule often associated with acne, which is a common skin condition characterized by clogged pores, bacterial infection, and inflammation.",
    "q_a": [
        {
            "question": "What condition is depicted in the image?",
            "answer": "The image depicts acne disease, showing inflammation and a raised lesion on the skin."
        },
        {
            "question": "What are the visible characteristics of the skin in the image?",
            "answer": "The visible characteristics include redness, raised lesions, inflammation, and enlarged pores."
        },
        {
            "question": "What type of lesion is likely shown in the image?",
            "answer": "The lesion appears to be a pustule or nodule, common types of acne lesions."
        },
        {
            "question": "What might be the causes of this skin condition?",
            "answer": "Causes of acne include clogged pores due to excess oil production, bacterial infection, hormonal fluctuations, or genetic predisposition."
        },
        {
            "question": "How can such acne be treated?",
            "answer": "Treatment can include topical creams, antibiotics, oral medication, proper skincare routines, and in severe cases, dermatological procedures like laser therapy or chemical peels."
        }
    ]
}



In [ ]:
# Example usage
image_path = 'images.jpeg'
data_url = local_image_to_data_url(image_path)

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        { "role": "system", "content": "You are a helpful assistant." },
        { "role": "user", "content": [
            {
                "type": "text",
                "text": "Describe this picture and create 5 questions with their answers about the description:"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": data_url
                }
            }
        ] }
    ],
    max_tokens=2000
)

print(response.choices[0].message.content)

### Description:
This image depicts the back of a person's head. The individual appears to have alopecia, a condition that causes hair loss in patches. The affected areas are prominent, with smooth areas of skin visible where hair is missing. The surrounding scalp has short hair, but the patches are completely bald. The background is neutral, with no distracting elements, emphasizing the focus on the head and the hair loss condition.

---

### Questions and Answers:

1. **Q: What condition does the person in the picture appear to have?**  
   **A:** The person appears to have alopecia, which causes hair loss in patches.

2. **Q: Where is the focus of the image?**  
   **A:** The focus of the image is on the back of the person's head, specifically on the bald patches caused by hair loss.

3. **Q: What does the skin in the bald patches look like?**  
   **A:** The skin in the bald patches appears smooth and free of hair.

4. **Q: How does the rest of the scalp surrounding the bald patche